In [1]:
# existing issues with data, to keep in mind...
# weather data is absent for ~12 hours at a time, often

In [2]:
import pandas as pd
import numpy as np
import glob, geocoder, time

In [3]:
# load, combine crime data
crimedf = pd.DataFrame(index=[],columns=[])
for counter, file in enumerate(glob.glob('raw_data/stl_crime_data/*.csv')):
    tmp = pd.read_csv(file, encoding = 'ISO-8859-1', skiprows=None, low_memory=False)
    tmp.columns=tmp.columns.str.replace('-','')
    tmp.columns=tmp.columns.str.replace(' ','')
    tmp.columns=tmp.columns.str.replace('DateOccured','Date')
    tmp.columns=tmp.columns.str.replace('DateOccur','Date')
    tmp.columns=tmp.columns.str.replace('ILEADSAddress','Address')
    tmp.columns=tmp.columns.str.replace('ILEADSStreet','Street')
    crimedf = pd.concat([crimedf,tmp], axis=0, ignore_index=True)
#crimedf = crimedf.sample(n=1000, axis=0)
crimedf = crimedf.reset_index().drop(['index'],1)

In [4]:
# clean crime data
dropcolumnscrime = ['CADAddress','CADStreet','AdministrativeAdjustmentIndicator','Beat','CodedMonth','Complaint','DateCrimeCoded','FlagAdministrative','FlagCleanup','FlagCrime','FlagUnfounded','ILeadsAdd','ILeadsApprove','ILeadsAsg','ILeadsType','LocationComment','LocationName','MonthReportedtoMSHP','NewCrimeIndicator','UnfoundedCrimeIndicator']
dropcolumnscrime = pd.Index(dropcolumnscrime).intersection(crimedf.columns).tolist()
crimedf = crimedf.drop(dropcolumnscrime, 1)
crimedf = crimedf[crimedf['Count']==1]
crimedf = crimedf.drop('Count', 1)
crimedf['Date'] = pd.to_datetime(crimedf['Date']) # to pandas datetime format
crimedf = crimedf[crimedf['Date'].dt.year>=2008]
# most of these cases correspond to no location (on some street, but no address or neighborhood)
crimedrop = ((crimedf['Address'].isnull()) | (crimedf['Street'].str.contains('UNKNOWN')) | (crimedf['Neighborhood']==0))
crimedf = crimedf[~crimedrop]
print('Dropped ',sum(crimedrop)/crimedf.shape[0]*100,'%',' of crimes',sep='')
strcols = ['Crime','District','Description','Address','Street']
crimedf[strcols] = crimedf[strcols].astype(str)
crimedf.loc[crimedf['Crime'].str.len()==5, 'Crime'] = '0' + crimedf.loc[crimedf['Crime'].str.len()==5, 'Crime']
crimedf = crimedf.reset_index().drop(['index'],1)

Dropped 0.5745492631106456% of crimes


In [5]:
# find lat/lon for crimes, use google lookup tool for empty data
import stateplane
from pyproj import Proj, transform

inProj = Proj(init='esri:102696', preserve_units = True)
outProj = Proj(init='epsg:4326')

dropinds = []
for ind in crimedf.index:
    xc = crimedf['XCoord'].iloc[ind]
    yc = crimedf['YCoord'].iloc[ind]
    if (xc==0):
        addrstring = crimedf['Address'].iloc[ind] + ' ' + crimedf['Street'].iloc[ind] + ' St. Louis, MO'
        print("Looking up w/ geocoder.google", addrstring, ind/crimedf.shape[0], sep=' ')
        g = geocoder.google(addrstring)
        if not g.latlng:
            crimedf.loc[ind,'XCoord'], crimedf.loc[ind,'YCoord'] = 906515.1600962189, 1017387.5860900703
            dropinds.append(ind)
        else:
            sp = stateplane.from_latlon(g.latlng[0],g.latlng[1])
            crimedf.loc[ind, 'XCoord'], crimedf.loc[ind, 'YCoord'] = sp[0]/0.3048, sp[1]/0.3048

crimedf['XCoord'], crimedf['YCoord'] = transform(inProj, outProj, crimedf['XCoord'].tolist(), crimedf['YCoord'].tolist())

print('Dropped ',len(dropinds)/crimedf.shape[0],'% of crimes',sep='')
crimedf = crimedf.drop(dropinds)
crimedf = crimedf.reset_index().drop(['index'],1)

Looking up w/ geocoder.google 400 4TH St. Louis, MO 3.3664995885389394e-05
Looking up w/ geocoder.google 2144 MARKET St. Louis, MO 4.301638363133089e-05
Looking up w/ geocoder.google 3526 WASHINGTON St. Louis, MO 9.912471010697987e-05
Looking up w/ geocoder.google 7080 JAMIESON St. Louis, MO 0.00011034637540210967
Looking up w/ geocoder.google 8000 HALL St. Louis, MO 0.00017954664472207674
Looking up w/ geocoder.google 5915 ERA St. Louis, MO 0.00019076831001720655
Looking up w/ geocoder.google     St. Louis, MO 0.00019637914266477146
Looking up w/ geocoder.google 3218 PORTIS St. Louis, MO 0.00021695219570584275
Looking up w/ geocoder.google 1535 MARKET St. Louis, MO 0.00024126580384529065
Looking up w/ geocoder.google 4521 MCMILLAN St. Louis, MO 0.00026557941198473853
Looking up w/ geocoder.google 1528 TEXAS St. Louis, MO 0.0004021096730754844
Looking up w/ geocoder.google 7834 RIO TINTO PL St. Louis, MO 0.0005012343831824643
Looking up w/ geocoder.google 3143 N. BROADWAY St. Louis, MO

Looking up w/ geocoder.google 4900 BERTHOLD AVE St. Louis, MO 0.021229520460836388
Looking up w/ geocoder.google 0 N. 11TH / WASHINGTON St. Louis, MO 0.021596094860477293
Looking up w/ geocoder.google 2500 CORA AVE St. Louis, MO 0.02161292735841999
Looking up w/ geocoder.google 0 I40 / MCCLAUSLAND St. Louis, MO 0.021740106231764793
Looking up w/ geocoder.google 5500 CABANNE AVE St. Louis, MO 0.021884117603052294
Looking up w/ geocoder.google 1100 HAMPTON AVE St. Louis, MO 0.02192152315403606
Looking up w/ geocoder.google 4300 ROSALIE ST St. Louis, MO 0.021955188149921447
Looking up w/ geocoder.google 3500 ARKANSAS AVE St. Louis, MO 0.022048702027380865
Looking up w/ geocoder.google 1500 CLARA AVE St. Louis, MO 0.02217214034562729
Looking up w/ geocoder.google 1700 S TUCKER BLVD St. Louis, MO 0.023112889952869007
Looking up w/ geocoder.google 4300 DR MARTIN LUTHER KING DR St. Louis, MO 0.0233373232587716
Looking up w/ geocoder.google 4306 GIBSON St. Louis, MO 0.023496296850452607
Lookin

Looking up w/ geocoder.google 3675 LOUGHBOROUGH St. Louis, MO 0.06408880077803546
Looking up w/ geocoder.google 3675 LOUGHBOROUGH St. Louis, MO 0.06409067105558465
Looking up w/ geocoder.google 5503 HODIAMONT St. Louis, MO 0.06410002244333059
Looking up w/ geocoder.google 1615 BROADWAY St. Louis, MO 0.06410376299842897
Looking up w/ geocoder.google 3400 OHIO AVE St. Louis, MO 0.06412246577392085
Looking up w/ geocoder.google 5250 PAGE St. Louis, MO 0.06416922271265056
Looking up w/ geocoder.google 4140 SHENANDOAH St. Louis, MO 0.06417483354529813
Looking up w/ geocoder.google 5600 LOTUS AVE St. Louis, MO 0.06418231465549487
Looking up w/ geocoder.google 2800 STODDARD ST St. Louis, MO 0.06419353632079
Looking up w/ geocoder.google 3014 N 25TH St. Louis, MO 0.0642103688187327
Looking up w/ geocoder.google 3800 FOLSOM AVE St. Louis, MO 0.06422533103912621
Looking up w/ geocoder.google 919 N JEFFERSON St. Louis, MO 0.06427582853295428
Looking up w/ geocoder.google 3015 N 23RD St. Louis, MO

Looking up w/ geocoder.google 2604 JEFFERSON St. Louis, MO 0.07264532056557194
Looking up w/ geocoder.google     St. Louis, MO 0.07266028278596544
Looking up w/ geocoder.google 1501 SWITZER St. Louis, MO 0.07267150445126057
Looking up w/ geocoder.google 1501 SWITZER St. Louis, MO 0.07267337472880976
Looking up w/ geocoder.google 5453 DELMAR BLVD St. Louis, MO 0.07267711528390813
Looking up w/ geocoder.google 1 N EUCLID AVE St. Louis, MO 0.07269020722675246
Looking up w/ geocoder.google 11 4TH St. Louis, MO 0.07270142889204757
Looking up w/ geocoder.google 5451 COLOGNE St. Louis, MO 0.07270329916959677
Looking up w/ geocoder.google 1217 BAYARD St. Louis, MO 0.07270516944714596
Looking up w/ geocoder.google 3742   St. Louis, MO 0.0727145208348919
Looking up w/ geocoder.google 2222 MARKET ST St. Louis, MO 0.07272761277773622
Looking up w/ geocoder.google 3325 LAWN St. Louis, MO 0.07274818583077729
Looking up w/ geocoder.google 1881 PINE ST St. Louis, MO 0.07275566694097405
Looking up w/ g

Looking up w/ geocoder.google   18TH CLARK St. Louis, MO 0.0904877683848283
Looking up w/ geocoder.google 4120   St. Louis, MO 0.0904896386623775
Looking up w/ geocoder.google 2613 IOWA St. Louis, MO 0.09050086032767263
Looking up w/ geocoder.google 4255 HAMPTON ST St. Louis, MO 0.09053826587865639
Looking up w/ geocoder.google 3300 E FARLIN AVE St. Louis, MO 0.09057567142964015
Looking up w/ geocoder.google 528 EILER ST St. Louis, MO 0.09064113114386176
Looking up w/ geocoder.google 300 S LENOR K SULLIVAN St. Louis, MO 0.09065983391935363
Looking up w/ geocoder.google 5901 WATERMAN St. Louis, MO 0.09067479613974713
Looking up w/ geocoder.google 1820 MARKET St. Louis, MO 0.09094411610683026
Looking up w/ geocoder.google 200 WELLS DR St. Louis, MO 0.09157814019600509
Looking up w/ geocoder.google 220 N TUCKER BLVD St. Louis, MO 0.09169970823670233
Looking up w/ geocoder.google 300 N. 8TH ST St. Louis, MO 0.091974639036433
Looking up w/ geocoder.google 1915 WASHINGTON AVE St. Louis, MO 0

...

Looking up w/ geocoder.google 3800 SAINT LOUIS AVE St. Louis, MO 0.9459134435550236
Looking up w/ geocoder.google 2100 E JOHN AVE St. Louis, MO 0.9459227949427695
Looking up w/ geocoder.google 3000 CALIFORNIA AVE St. Louis, MO 0.9459246652203187
Looking up w/ geocoder.google 5700 GOODFELLOW PL St. Louis, MO 0.9459433679958106
Looking up w/ geocoder.google 4215 MARGARETTA St. Louis, MO 0.9459583302162041
Looking up w/ geocoder.google 3989 MERAMEC St. Louis, MO 0.9459602004937533
Looking up w/ geocoder.google 1108 SARAH St. Louis, MO 0.9459658113264008
Looking up w/ geocoder.google 4550 MCPHERSON St. Louis, MO 0.9459789032692452
Looking up w/ geocoder.google 3500 CALIFORNIA AVE St. Louis, MO 0.9459826438243435
Looking up w/ geocoder.google 5743 WELLS St. Louis, MO 0.946006957432483
Looking up w/ geocoder.google 2644 GRAVOIS St. Louis, MO 0.9460406224283684
Looking up w/ geocoder.google 5422 ROBIN St. Louis, MO 0.9460817685344505
Looking up w/ geocoder.google 4200 W KOSSUTH AVE St. Louis,

Looking up w/ geocoder.google 1521 E GRAND AVE St. Louis, MO 0.9545616069424703
Looking up w/ geocoder.google 1523 E. GRAND St. Louis, MO 0.9545840502730605
Looking up w/ geocoder.google 1901 SAINT LOUIS AVE St. Louis, MO 0.9545896611057081
Looking up w/ geocoder.google 2501 N. GRAND St. Louis, MO 0.9547336724769956
Looking up w/ geocoder.google 2501 N GRAND BLVD St. Louis, MO 0.9547355427545448
Looking up w/ geocoder.google 2501 N GRAND BLVD St. Louis, MO 0.9547374130320939
Looking up w/ geocoder.google 6318 WEST FLORISSANT St. Louis, MO 0.9550534899379067
Looking up w/ geocoder.google 6142   St. Louis, MO 0.955105857709284
Looking up w/ geocoder.google 0 I 70 WESTBOUND AT CA St. Louis, MO 0.9552797935213586
Looking up w/ geocoder.google 0 I 70 WESTBOUND AT CA St. Louis, MO 0.9552816637989078
Looking up w/ geocoder.google 0 I 70 WESTBOUND AT CA St. Louis, MO 0.9552835340764569
Looking up w/ geocoder.google 0 I 70 WESTBOUND AT CA St. Louis, MO 0.9552854043540061
Looking up w/ geocoder.

Looking up w/ geocoder.google 6142 WEST FLORISSANT AVE. St. Louis, MO 0.961799581057829
Looking up w/ geocoder.google 4615 SOUTH GRAND St. Louis, MO 0.9623251290491509
Looking up w/ geocoder.google 520 S JEFFERSON AVE St. Louis, MO 0.9623288696042492
Looking up w/ geocoder.google 2510 MCCAUSLAND AVE St. Louis, MO 0.9625981895713324
Looking up w/ geocoder.google 8959 RIVERVIEW BLVD St. Louis, MO 0.962633724844767
Looking up w/ geocoder.google 800 WASHINGTON AVE St. Louis, MO 0.9634435550235655
Looking up w/ geocoder.google 5209 WATERMAN BLVD St. Louis, MO 0.9636904316600583
Looking up w/ geocoder.google 5225 WALSH ST St. Louis, MO 0.9652277998054911
Looking up w/ geocoder.google 1220 N BROADWAY St. Louis, MO 0.9652296700830403
Looking up w/ geocoder.google 1400 RUSSELL St. Louis, MO 0.9658842672252562
Looking up w/ geocoder.google 6169 DES PERES AVE St. Louis, MO 0.9661143113638064
Looking up w/ geocoder.google 8411 HALLS FERRY St. Louis, MO 0.9665313832572754
Looking up w/ geocoder.goo

Looking up w/ geocoder.google 600 OLIVE ST St. Louis, MO 0.973241939103763
Looking up w/ geocoder.google 159 RIVER CITY CASINO BLVD St. Louis, MO 0.97334106381387
Looking up w/ geocoder.google 1400 LUCAS AVE St. Louis, MO 0.9733448043689683
Looking up w/ geocoder.google 1900 PINE ST St. Louis, MO 0.9734570210219197
Looking up w/ geocoder.google 1500 HAMPTON AVE St. Louis, MO 0.973500037405551
Looking up w/ geocoder.google 4600 FOREST PARK AVE St. Louis, MO 0.9735131293483953
Looking up w/ geocoder.google 500 CONVENTION PZ St. Louis, MO 0.9736178648911499
Looking up w/ geocoder.google 4100 S BROADWAY St. Louis, MO 0.9737113787686092
Looking up w/ geocoder.google 1 THEATRE DR St. Louis, MO 0.9737936709807735
Looking up w/ geocoder.google 3400 HAMILTON AVE St. Louis, MO 0.973817984588913
Looking up w/ geocoder.google 0 N 3RD ST / EADS BRDG St. Louis, MO 0.9738610009725444
Looking up w/ geocoder.google 0 N 17TH ST / ST CHARLES ST St. Louis, MO 0.9739002768010773
Looking up w/ geocoder.goog

Looking up w/ geocoder.google 2301 JEFFERSON St. Louis, MO 0.9841456572155308
Looking up w/ geocoder.google 2206 FARRAR St. Louis, MO 0.9841643599910227
Looking up w/ geocoder.google 3512 OSAGE St. Louis, MO 0.9841830627665146
Looking up w/ geocoder.google 100 GARRISON St. Louis, MO 0.9841998952644573
Looking up w/ geocoder.google 4332 PENROSE St. Louis, MO 0.9842204683174983
Looking up w/ geocoder.google 7933 BROADWAY St. Louis, MO 0.9842279494276951
Looking up w/ geocoder.google 3500 INDIANA St. Louis, MO 0.9842803171990724
Looking up w/ geocoder.google 3800 HEBERT ST St. Louis, MO 0.9842859280317199
Looking up w/ geocoder.google 6340 SOUTHWOOD AVE St. Louis, MO 0.9846020049375327
Looking up w/ geocoder.google 4044 N. GRAND St. Louis, MO 0.9847852921373532
Looking up w/ geocoder.google 4124 W. FLORISSANT St. Louis, MO 0.9849012493454029
Looking up w/ geocoder.google 98 FOREST PARK AVE St. Louis, MO 0.985260342634847
Looking up w/ geocoder.google 3916 N FLORISSANT AVE St. Louis, MO 0.

In [6]:
# load and clean weather data
weatherdf = pd.read_csv('raw_data/stl_weather/stl_weather_hourly.csv', encoding = 'ISO-8859-1', skiprows=None, low_memory=False)
dropcolumnsweather = ['PeakWindDirection','HOURLYAltimeterSetting','PeakWindDirection','DAILYAverageSeaLevelPressure','STATION','STATION_NAME','ELEVATION','LATITUDE','LONGITUDE','HOURLYDRYBULBTEMPC','HOURLYWETBULBTEMPC','HOURLYDewPointTempC','HOURLYPressureTendency','HOURLYPressureChange','HOURLYSeaLevelPressure']    
weatherdf = weatherdf.drop(dropcolumnsweather, 1)
monthlydailycols = weatherdf.filter(regex='Monthly|monthly|DAILY').columns
weatherdf = weatherdf.drop(monthlydailycols, 1)
weatherdf = weatherdf[weatherdf['REPORTTPYE']!='SOD'].drop(['REPORTTPYE'],1)
weatherdf['DATE'] = pd.to_datetime(weatherdf['DATE'])
weatherdf['HOURLYDRYBULBTEMPF'] = weatherdf['HOURLYDRYBULBTEMPF'].str.replace('s','')
weatherdf['HOURLYVISIBILITY'] = weatherdf['HOURLYVISIBILITY'].str.replace('V','')
weatherdf['HOURLYDewPointTempF'] = weatherdf['HOURLYDewPointTempF'].str.replace('s','')
weatherdf['HOURLYWindSpeed'] = weatherdf['HOURLYWindSpeed'].str.replace('s','')
weatherdf['HOURLYStationPressure'] = weatherdf['HOURLYStationPressure'].str.replace('s','')
weatherdf['HOURLYPrecip'] = weatherdf['HOURLYPrecip'].str.replace('s','').str.replace('T','.01')
weatherdf['HOURLYSKYCONDITIONS'] = weatherdf['HOURLYSKYCONDITIONS'].str[0:3]
weatherdf.loc[weatherdf['HOURLYSKYCONDITIONS'].isin(['BKN','SCT','FEW','OVC','CLR'])==False,'HOURLYSKYCONDITIONS'] = 'OTH'
weatherdf = weatherdf[~weatherdf['DATE'].isnull()]
# make numeric & fill in nan values with previous readings
numcols = ['HOURLYVISIBILITY','HOURLYPRSENTWEATHERTYPE','HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYWindGustSpeed','HOURLYStationPressure','HOURLYPrecip']
weatherdf[numcols] = weatherdf[numcols].apply(pd.to_numeric, errors='coerce', axis=1)
weatherdf = weatherdf.fillna(method='ffill')
weatherdf = weatherdf.fillna(method='bfill')
weatherdf = weatherdf.reset_index().drop(['index'],1)

In [7]:
# make weather columns in crime dataframe
def find_nearest(array,value):
    ind = (np.abs(array-value)).argmin()
    return ind

crimedftmp = pd.DataFrame(columns=['HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYVISIBILITY','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYStationPressure','HOURLYPrecip','HOURLYSKYCONDITIONS'])
start = time.time()
for ind in crimedf.index:
    minind = find_nearest(weatherdf['DATE'], crimedf.loc[ind,'Date'])
    weatherdfminind = weatherdf.loc[minind,:]
    crimedftmp = crimedftmp.append(weatherdfminind[['HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYVISIBILITY','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYStationPressure','HOURLYPrecip','HOURLYSKYCONDITIONS']] )
    print(ind/crimedf.shape[0], sep=' ', end="\r")

end = time.time()
crimedftmp = crimedftmp.reset_index().drop('index',1)
crimedf = pd.merge(crimedf,crimedftmp, left_index=True, right_index=True)
crimedf.columns = ['Address', 'Crime', 'Date', 'Description', 'District', 'Neighborhood','Street', 'XCoord', 'YCoord', 'dry_temp','wet_temp','visibility','dew_point','humidity','wind_speed','wind_dir','pressure','precip','sky']
print('\n')
print('Took', (end-start)/3600, 'hours',sep=' ')

/Users/Maczilla/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  This is separate from the ipykernel package so we can avoid doing imports until


0.99999811344959516454

Took 3.842772493097517 hours


In [8]:
# Load neighborhood population data
hooddict = {'Carondelet': 1, 'Patch': 2, 'Holly Hills': 3, 'Boulevard Heights': 4, 'Bevo Mill': 5, 'Princeton Heights': 6, 'South Hampton': 7, 'St. Louis Hills': 8, 'Lindenwood Park': 9, 'Ellendale': 10, 'Clifton Heights': 11, 'The Hill': 12, 'Southwest Garden': 13, 'North Hampton': 14, 'Tower Grove South': 15, 'Dutchtown': 16, 'Mount Pleasant': 17, 'Marine Villa': 18, 'Gravois Park': 19, 'Kosciusko': 20, 'Soulard': 21, 'Benton Park': 22, 'McKinley Heights': 23, 'Fox Park': 24, 'Tower Grove East': 25, 'Compton Heights': 26, 'Shaw': 27, 'Botanical Heights (formerly McRee Town)': 28, 'Tiffany': 29, 'Benton Park West': 30, 'The Gate District': 31, 'Lafayette Square': 32, 'Peabody, Darst, Webbe': 33, 'La Salle': 34, 'Downtown': 35, 'Downtown West': 36, 'Midtown': 37, 'Central West End': 38, 'Forest Park Southeast': 39, 'Kings Oak': 40, 'Cheltenham': 41, 'Clayton-Tamm': 42, 'Franz Park': 43, 'Hi-Pointe': 44, 'Wydown-Skinker': 45, 'Skinker-DeBaliviere': 46, 'DeBaliviere Place': 47, 'West End': 48, 'Visitation Park': 49, 'Wells-Goodfellow': 50, 'Academy': 51, 'Kingsway West': 52, 'Fountain Park': 53, 'Lewis Place': 54, 'Kingsway East': 55, 'The Greater Ville': 56, 'The Ville': 57, 'Vandeventer': 58, 'JeffVanderLou': 59, 'St. Louis Place': 60, 'Carr Square': 61, 'Columbus Square': 62, 'Old North St. Louis': 63, 'Near North Riverfront': 64, 'Hyde Park': 65, 'College Hill': 66, 'Fairground Neighborhood': 67, "O'Fallon": 68, 'Penrose': 69, 'Mark Twain-I-70 Industrial': 70, 'Mark Twain': 71, 'Walnut Park East': 72, 'North Point': 73, 'Baden': 74, 'Riverview': 75, 'Walnut Park West': 76, 'Covenant Blu-Grand Center': 77, 'Hamilton Heights': 78, 'North Riverfront': 79, 'Carondelet Park': 80, 'Tower Grove Park': 81, 'Forest Park': 82, 'Fairgrounds Park': 83, 'Penrose Park': 84, 'Calvary-Bellefontaine Cemetaries': 86}
inv_hooddict = {v: k for k, v in hooddict.items()}
hoodpopdf = pd.read_csv('raw_data/StL_neighborhood_populations_2010.csv', encoding = 'ISO-8859-1', skiprows=None)
hoodpopdf['Neighborhood'].replace(hooddict, inplace=True)

In [9]:
# Save as cleaned "pickle" dataframe files
crimedf.to_pickle('cleaned_data/crimedf.pkl')
hoodpopdf.to_pickle('cleaned_data/hoodpopdf.pkl')
weatherdf.to_pickle('cleaned_data/weatherdf.pkl')

In [10]:
crimedf['Crime'].value_counts()

142320    51203
265321    27779
065701    26556
064701    24027
071013    20295
091113    19977
067701    17222
266999    16414
091114    14821
064601    14791
051312    13872
063701    12209
041011    11952
241100    11897
069702    11781
051112     9544
031111     9255
071030     9004
064711     8759
067601     8520
069602     7704
182400     7492
266510     7101
051322     6232
182130     5595
065601     5501
071040     5290
182250     4517
031421     4353
250000     4227
          ...  
266210        1
061501        1
243100        1
182110        1
113000        1
103100        1
034212        1
042202        1
044105        1
037422        1
067600        1
184120        1
181380        1
044205        1
266220        1
152120        1
175200        1
033211        1
037212        1
174110        1
192500        1
033212        1
042206        1
061711        1
034312        1
053321        1
037411        1
043105        1
042101        1
065501        1
Name: Crime, Length: 458